# CAO POINTS ANALYSIS
### AUTHOR: ANTE DUJIC
<hr style="border:2px solid black"> </hr>

## INTRODUCTION
<hr style="border:2px solid gray"> </hr>

This notebook gives an overview of how to load CAO points information from the CAO website into a pandas data frame and the comparison of CAO points in 2019, 2020, and 2021.

[<center><img src="http://www.cao.ie/images/cao.png" width="100"/></center> ](http://www.cao.ie/index.php)

***
### CONTENTS

- [WHAT IS CAO](#CAO)
- [LEVEL 8 - ROUND 1 and ROUND 2](#R1R2)
    - [THE ORIGINAL DATA STRUCTURE]()
    - [SCRAPPING AND CLEANING THE DATA]()
    - [CLEANED DATA]()
- [LEVEL 8 - EOS and MID]()
    - [THE ORIGINAL DATA STRUCTURE]()
    - [SCRAPPING AND CLEANING THE DATA]()
    - [CLEANED DATA]()
2. [CONCATENATING THE DATA](#CONCATENATE)

## WHAT IS CAO? <a id='CAO'></a>
<hr style="border:2px solid gray"> </hr>

The purpose of the Central Applications Office (CAO) is to process centrally applications for undergraduate courses in Irish Higher Education Institutions (HEIs), and to deal with them in an efficient and fair manner. [1]

Students applying for admission to third level education courses in Ireland apply to the CAO rather than to individual educational institutions such as colleges and universities. The CAO then offers places to students who meet the minimum requirements for a course for which they have applied. If for a particular course there are more qualified applicants than available places, the CAO makes offers to those applicants with the highest score in the CAO points system. If students do not accepts offers, or later decline them because they receive an offer for another course, the CAO makes further offers until all of the places have been filled or until the offer season closes. [2]

In [1]:
# HTTP request
import requests as rq

# Regular expressions
import re

# Dates and time
import datetime as dt

# Data frames
import pandas as pd

# For downloading
import urllib.request as urlrq

# PDF
import camelot

# To use .unescape
import html

# Arrays
import numpy as np

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

<br>
Current data and time are used for the filenames that will get created to save the original data and the cleaned data. I've used the date and time for easier access and organization of the data, but also to avoid the overwriting of the data files.

In [2]:
# Current date and time
now = dt.datetime.now()
# Format as a string
nowstr = (now.strftime("%Y%m%d_%H%M%S"))

## LEVEL 8 - ROUND 1 and ROUND 2
<hr style="border:2px solid gray"> </hr>

The data for Round 1 and Round 2 for the three given years (2019., 2020., 2021.) has been pulled from the CAO website. As mentioned in the README, CAO practice is to first upload the data with only Round 1 and Round 2 points and then overwrite that data with EOS and MID points after the academic year starts. To obtain the Round 1 and Round 2 data for the 2019. and 2020. I have used [The Internet Archive](https://web.archive.org/). This didn't only allow me access to the archived version of the website but also gave me the same format and the structure of the data for all the years.

### THE ORIGINAL DATA STRUCTURE
***

Original data: <br>
2019: https://web.archive.org/web/20191019135815/http://www2.cao.ie:80/points/l8.php <br>
2020: https://web.archive.org/web/20201108133105/http://www2.cao.ie/points/l8.php <br>
2021: http://www2.cao.ie/points/l8.php

The data for Round 1 and Round 2 points on the CAO website has the following structure:
- Title
- Information on how to read the data
- List of colleges
- Title
- Points
    - Course code
    - Course title
    - Round 1 points
    - Round 2 points

The data of interest is contained under the *Points* section of the website. It is explained below how the data was scrapped, cleaned and saved for the later analysis.

### SCRAPPING AND CLEANING THE DATA
***

<br>
To loop through the years and run multiple lines of code for each year in one go, I've created the dictionary below.

In [3]:
# Defining a dictionary
years_dict = {
    "2019": [("data/cao2019" +  nowstr), "https://web.archive.org/web/20191019135815/http://www2.cao.ie:80/points/l8.php"],
    "2020": [("data/cao2020" +  nowstr), "https://web.archive.org/web/20201108133105/http://www2.cao.ie/points/l8.php"],
    "2021": [("data/cao2021" +  nowstr), "http://www2.cao.ie/points/l8.php"]
}

STEPS:
<br>
1. Save the original html
<br>
This was done to ensure the original website format from which the data was scrapped is kept, in case of any future changes on the CEO website itself. The data was fetched from the given url-s and it was encoded (cp1252), due to symbols in certain course names not being supported. NOTE: Encoding on the website is wrong.
<br>
2. Read the data and filter out only the relevant data
<br>
The fetched data was ittered through and filtered out using the below regular expression. Regular expression is a sequence of characters that specifies a search pattern. Usually such patterns are used by string-searching algorithms for "find" or "find and replace" operations on strings, or for input validation. [4] The set regular expression filters out only the lines starting with the course code (e.g. AL801). Also, Symbol "&" instead "and" was used in certain 2019 and 2020 course names and it was being decoded as "&amp;" when saving to csv. To avoid this *html.unescape* function was used. The data was also structured to fit the csv file - split and divided by commas to create 4 columns "CODE", "TITLE", "R1_POINTS", "R2_POINTS".

In [4]:
# Regular expression
re_course = re.compile(r"([A-Z]{2}[0-9]{3})(.*)")

3. Save the cleaned data as csv files

In [5]:
# Loop through the (dict) years
for year, content in years_dict.items():
    # Fetch the CAO points URL
    rq.get (content[1])
    resp = rq.get (content[1])
    # The server uses the wrong encoding
        # Change to "cp1252"
    resp.encoding = "cp1252"
    # Check if OK:
        #Response [200] means OK
    print (year,resp)
    
    # Save the original html file
    with open(content[0] + ".html", "w") as f:
        f.write(resp.text)
        # Keep track of how many courses we process
    
    no_lines = 0
    # Iterating through the lines
    resp.iter_lines()
    resps = resp.iter_lines()
    
    # Open the csv file for writing (saving)
    with open(content[0] + ".csv", "w") as f:
        # Cleaning the data
        # Write a header row.
        f.write(','.join(["CODE", "TITLE", "R1_POINTS", "R2_POINTS"]) + "\n")
        # Loop through lines of the response.
        for line in resps:
            # Decode the line, using the wrong encoding
            dline = line.decode("cp1252")
            # Converting &amp; to &
            dline = html.unescape(dline)
            # Match only the lines representing courses
            if re_course.fullmatch(dline):
                # Add one to the lines counter
                no_lines = no_lines + 1
                # The course code
                course_code = dline[:5]
                # The course title
                course_title = dline[7:57]
                # Round one points
                course_points = re.split(' +', dline[60:])
                if len(course_points) != 2:
                    course_points = course_points[:2]
                # Join the fields using a comma
                linesplit = [course_code, course_title, course_points[0], course_points[1]]
                # Rejoin the substrings with commas in between
                f.write(",".join(linesplit) + "\n")   
    # Print the total number of processed lines
    print("Total number of lines in CAO", year, "database is", no_lines)

2019 <Response [200]>
Total number of lines in CAO 2019 database is 930
2020 <Response [200]>
Total number of lines in CAO 2020 database is 961
2021 <Response [200]>
Total number of lines in CAO 2021 database is 949


### CLEANED DATA
***

#### 2019
***

In [6]:
# Read csv file
df_2019 = pd.read_csv ("data/cao2019" +  nowstr + ".csv", encoding = "cp1252")
df_2019.columns = ["CODE","TITLE", "R1_2019", "R2_2019"]
# Output random 10 rows
df_2019.sample(n = 10)

,CODE,TITLE,R1_2019,R2_2019
181,DB504,Computing (Web and Mobile Technology) ...,265,227
177,DB500,Computing ...,242,NaN
840,SG147,Sport with Business ...,269,NaN
678,LY608,Mechanical Engineering ...,304,NaN
752,LM020,Law and Accounting ...,436,NaN
623,GY128,Arts (Global Languages) ...,370,NaN
22,AL860,Social Care Practice ...,310,309
609,GY111,Arts with Creative Writing ...,402,NaN
803,MH202,Biotechnology ...,429,NaN
259,DC238,Social Sciences and Cultural Innovation ...,358,NaN


#### 2020
***

In [7]:
# Read csv file
df_2020 = pd.read_csv ("data/cao2020" +  nowstr + ".csv", encoding = "cp1252")
df_2020.columns = ["CODE","TITLE", "R1_2020", "R2_2020"]
# Output random 10 rows
df_2020.sample(n = 10)

,CODE,TITLE,R1_2020,R2_2020
332,TU802,Electrical Services and Energy Management ...,317,300
575,DK887,Environmental Science ...,330,NaN
337,TU807,Computer Engineering in Mobile Systems ...,276,NaN
577,GA182,Business ...,287,NaN
531,DN452,Midwifery ...,489,476
955,WD195,Architectural and Building Information Modelli...,279,NaN
67,CR108,Mechanical Engineering ...,434,NaN
745,LC252,Civil Engineering Management ...,270,NaN
307,ID001,Accounting and Finance ...,203,NaN
261,DC232,Law and Society (BCL) ...,466,NaN


#### 2021
***

In [8]:
# Read csv file
df_2021 = pd.read_csv ("data/cao2021" +  nowstr + ".csv", encoding = "cp1252")
df_2021.columns = ["CODE","TITLE", "R1_2021", "R2_2021"]
# Output random 10 rows
df_2021.sample(n = 10)

,CODE,TITLE,R1_2021,R2_2021
406,TU942,Culinary Arts ...,252,NaN
471,TR062,Geography and Geoscience ...,495,NaN
823,MH207,Data Science ...,444,NaN
239,DC166,Environmental Science and Technology ...,423*,NaN
810,MH002,Education - Primary Teaching - Gaeltacht Appli...,NaN,NaN
117,CK109,English - 3 years or 4 years (International Pa...,367,357
98,CR500,Engineering (Common Entry) ...,475,NaN
35,CW068,Applied Social Studies in Professional Social ...,250,NaN
656,GY320,Physics (Degree options: Applied Astrophysics ...,465,NaN
921,WD091,Hospitality Management ...,225,NaN


## 2. LEVEL 8 - EOS and MID
<hr style="border:2px solid gray"> </hr>

As mentioned in README, EOS and MID points data becomes available after the start of the academic year, and are only available for 2019. and 2020. in the time of making this project. However, the project could be updated in the future when the data becomes available for 2021.

### 2019 - THE PDF FILE STRUCTURE
***

Original data: <br>
2019: http://www2.cao.ie/points/lvl8_19.pdf <br>

Data containing EOS and MID points for 2019 is in pdf format.
The file has the following structure:
- Title and the Subtitles of the file
- Information on the file
- Information on how to read the data
- Points table
    - Course code
    - Institution and Course
    - EOS points
    - MID points

The data of interest is contained under the table section of the file. It is explained below how the data was scrapped, cleaned and saved for the later analysis.

### SCRAPPING AND CLEANING THE DATA
***

STEPS:
<br>
1. Save the original file <br>
Same as with the R1 and R2 points, I've saved the original file locally. The data is fetched from the given url using *requests* library.

In [9]:
# Creating a file path for the original data
path_pdf = 'data/cao2019_eos' + nowstr + '.pdf'

In [10]:
# Fetch the CAO points URL
resp_pdf = rq.get("http://www2.cao.ie/points/lvl8_19.pdf")
resp_pdf # <Response [200]> means OK

<Response [200]>

In [11]:
# Save the original file
with open(path_pdf, 'wb') as f:
    f.write(resp_pdf.content)


2. Read the pdf file and filter out the table only 
<br>
The saved data is read using the *camelot* libray.

In [12]:
# Read the pdf file
tables = camelot.read_pdf(path_pdf, pages = "all", flavor = "lattice")


3. Concatenate the tables read 
<br>
Camelot reads the file and returns a table for each page. It is not possible to return one table from all the pages. Therefore, all the tables have to be concatenated.
<br>
4. Save the data as csv file 
<br>
The concatenated table is saved as csv file.

In [13]:
# Check the total number of tables read to decide the loop range
print ("Tables:", tables.n)

Tables: 18


In [14]:
# Create empty list
table_total = []
# Loop through all 18 tables
for x in range (0,18):
    df = tables[x].df
    # Append all 18 tables to table_total
    table_total.append(df)

# Concatenate all tables  
table = pd.concat(table_total)
# Remove old column names
table = table.iloc[1: , :]
# Name the columns
table.columns = ["CODE","TITLE", "EOS_2019", "MID_2019"]
# Sort table by "CODE" column
    # to remove irrelevant data (college names)
table.sort_values("CODE", inplace = True)
# Remove first 35 rows (college names)
table = table.iloc[35: , :]
# Save .csv file
table.to_csv("data/cao2019_eos" +  nowstr + ".csv", index = False)

### CLEANED DATA
***

In [15]:
# Read csv file
df_em_2019 = pd.read_csv ("data/cao2019_eos" +  nowstr + ".csv", encoding = "cp1252")
# Output random 10 rows
df_em_2019.sample(n = 10)

,CODE,TITLE,EOS_2019,MID_2019
411,DT343,Retail and Services Management,301,332.0
165,CR510,Sustainable Energy Engineering,319,343.0
464,GA482,Quantity Surveying and Construction Economics,280,340.0
406,DT299,Chemical Sciences with Medicinal Chemistry,420,445.0
208,CW938,Business with Law,270,288.0
899,WD027,Music,#290,317.0
299,DK816,Business Studies,300,357.0
381,DT066,Engineering (General Entry),400,457.0
822,TL845,Civil Engineering,300,364.0
115,CK702,Dentistry,590,613.0


### 2020 - THE XLSX FILE STRUCTURE
***

Original data: <br>
2020: http://www2.cao.ie/points/CAOPointsCharts2020.xlsx

Data containing EOS and MID points for 2020 is in xlsx format. It contains the data for LEVEL 6, 7 and 8. The file has the following structure:
The file has the following structure:
- Title and the Subtitles of the file
- Information on the data in the file
- Information on how to read the data
- Points table


The data of interest is contained under the columns *COURSE TITLE, COURSE CODE2, EOS* and *EOS Mid-point*. It is explained below how the data was scrapped, cleaned and saved for the later analysis.

### SCRAPPING AND CLEANING THE DATA
***

STEPS:
<br>
1. Save the original file <br>
To fetch the data from the pdf file I've used the *urllib.request* library. It is fetched from the given url and saved locally.

In [16]:
# Create a file path for the original data
path_xlsx = ("data/cao2020_eos" +  nowstr + ".xlsx")

# Copying network xlsx file
urlrq.urlretrieve('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', path_xlsx)

('data/cao2020_eos20211207_215124.xlsx',
 <http.client.HTTPMessage at 0x1f453989f70>)

2. Read the xlsx file and clear the data <br>
File contains irrelevant data in the first 10 rows, which I've ignored when reading the file. I've then filtered out the relevant data from the table.

In [17]:
# Download and parse the excel spreadsheet
df2020_eos = pd.read_excel("http://www2.cao.ie/points/CAOPointsCharts2020.xlsx", skiprows = 10)

# Filter out only level 8 courses
df2020_eos = df2020_eos.loc[df2020_eos["LEVEL"] == 8]
# Filter out only relevant columns
df2020_eos = df2020_eos [["COURSE CODE2", "COURSE TITLE", "EOS", "EOS Mid-point"]]
# Change column names
df2020_eos.columns = ["CODE","TITLE", "EOS_2020", "MID_2020"]


3. Save the data as csv file

In [18]:
# Save pandas data frame
df2020_eos.to_csv("data/cao2020_eos" +  nowstr + ".csv", index = False)

### CLEANED DATA
***

In [19]:
# Read csv file
df_em_2020 = pd.read_csv ("data/cao2020_eos" +  nowstr + ".csv")
# Output random 10 rows
df_em_2020.sample(n = 10)

,CODE,TITLE,EOS_2020,MID_2020
292,DK832,Construction Management,387,419
608,LM152,Mental Health Nursing,419,430
543,LC320,Business,346,388
312,DK884,Sustainable Agriculture,NaN,NaN
128,CR125,Popular Music: Electric Bass Guitar at CIT Cor...,1088,1088
621,LY708,Applied Computing,306,372
865,TR664,Religion/English Literature,498,565
471,GY202,Commerce (International) with French,484,511
975,TU974,Fine Art,563,826
316,DL821,"English, Media and Cultural Studies",212,347


## 3. CONCATENATING THE DATA <a id='CONCATENATE'></a>
<hr style="border:2px solid gray"> </hr>

After cleaning and saving all the data, I've merged them all into one table.

STEPS:
1. Concatenating all the dataframes into one table <br>
This results with a table containing ALL the data from the listed dataframes, meaning the same courses from each table get repeated. 

In [20]:
# Defining a list containing all the saved csv files
df_list = [df_2019, df_em_2019, df_2020, df_em_2020, df_2021]
# Concatenating the data contained within a list
allcourses = pd.concat(df_list)

2. Cleaning the concatenated table <br>
I've filtered out only the common columns, *CODE* and *TITLE*, to create a table containing all the possible courses from each year. This table will be a base on which I will later add the relevant data. I've also removed all the duplicates, sort the values by the *CODE* column and set the same column as the index.


In [21]:
# Filering out columns
allcourses = allcourses [["CODE", "TITLE"]]
# Remove duplicates created concatenating
allcourses.drop_duplicates(subset=["CODE"], inplace=True, ignore_index=False)
# Sort the table by "CODE" column
allcourses.sort_values("CODE", inplace = True)
# set "CODE" column as index 
allcourses.set_index("CODE", inplace=True)
# Print only first 10 rows for an overview
allcourses.head(10)

,TITLE
CODE,
AC120,International Business ...
AC137,Liberal Arts ...
AD101,First Year Art & Design (Common Entry) ...
AD102,Graphic Design and Moving Image Design ...
AD103,Textile & Surface Design and Jewellery & Objec...
AD202,Education & Design or Fine Art (Second Level T...
AD204,Fine Art ...
AD211,Fashion Design ...
AD212,Product Design ...


3. Joining the relevant columns to the "base" table

In [22]:
# Loop - set "CODE" column as index for all df
for i in df_list:
    i.set_index("CODE", inplace=True)

allcourses = allcourses.join(df_2019[["R1_2019", "R2_2019"]])
allcourses = allcourses.join(df_em_2019[["EOS_2019", "MID_2019"]])
allcourses = allcourses.join(df_2020[["R1_2020", "R2_2020"]])
allcourses = allcourses.join(df_em_2020[["EOS_2020", "MID_2020"]])
allcourses = allcourses.join(df_2021[["R1_2021", "R2_2021"]])

allcourses.head(10)

,TITLE,R1_2019,R2_2019,EOS_2019,MID_2019,R1_2020,R2_2020,EOS_2020,MID_2020,R1_2021,R2_2021
CODE,,,,,,,,,,,
AC120,International Business ...,234,234,234,269.0,209,NaN,209,280,294,294
AC137,Liberal Arts ...,275,252,252,275.0,252,NaN,252,270,271,270
AD101,First Year Art & Design (Common Entry) ...,#+Matric,NaN,# +matric 550,NaN,#+matric,NaN,#+matric,#+matric,#554,NaN
AD102,Graphic Design and Moving Image Design ...,#+Matric,NaN,# +matric 635,NaN,#+matric,NaN,#+matric,#+matric,#538,NaN
AD103,Textile & Surface Design and Jewellery & Objec...,#+Matric,NaN,# +matric 545,NaN,#+matric,NaN,#+matric,#+matric,#505,NaN
AD202,Education & Design or Fine Art (Second Level T...,#+Matric,NaN,# +matric 580,NaN,#+matric,NaN,#+matric,#+matric,#591,NaN
AD204,Fine Art ...,#+Matric,NaN,# +matric 600,NaN,#+matric,NaN,#+matric,#+matric,#514,NaN
AD211,Fashion Design ...,#+Matric,NaN,# +matric 600,NaN,#+matric,NaN,#+matric,#+matric,#760,#679
AD212,Product Design ...,#+Matric,NaN,# +matric 600,NaN,#+matric,NaN,#+matric,#+matric,#413,NaN


4. Saving the final table as a csv file

In [23]:
allcourses.to_csv ("data/Final_table.csv")

## 4. DATA ANALYSIS
<hr style="border:2px solid gray"> </hr>

In [24]:
df = pd.read_csv ("data/Final_table.csv", index_col = ["CODE", "TITLE"])
df

,,R1_2019,R2_2019,EOS_2019,MID_2019,R1_2020,R2_2020,EOS_2020,MID_2020,R1_2021,R2_2021
CODE,TITLE,,,,,,,,,,
AC120,International Business,234,234,234,269.0,209,NaN,209,280,294,294
AC137,Liberal Arts,275,252,252,275.0,252,NaN,252,270,271,270
AD101,First Year Art & Design (Common Entry),#+Matric,NaN,# +matric 550,NaN,#+matric,NaN,#+matric,#+matric,#554,NaN
AD102,Graphic Design and Moving Image Design,#+Matric,NaN,# +matric 635,NaN,#+matric,NaN,#+matric,#+matric,#538,NaN
AD103,Textile & Surface Design and Jewellery & Objects,#+Matric,NaN,# +matric 545,NaN,#+matric,NaN,#+matric,#+matric,#505,NaN
...,...,...,...,...,...,...,...,...,...,...,...
WD211,Creative Computing,275,NaN,275,322.0,271,NaN,271,318,270,NaN
WD212,Recreation and Sport Management,274,NaN,274,311.0,270,NaN,270,349,262,NaN
WD230,Mechanical and Manufacturing Engineering,273,NaN,273,348.0,253,NaN,253,369,230,230


In [25]:
df.dtypes

R1_2019      object
R2_2019      object
EOS_2019     object
MID_2019    float64
R1_2020      object
R2_2020      object
EOS_2020     object
MID_2020     object
R1_2021      object
R2_2021      object
dtype: object

In [26]:
# Filtering out the non numeric values in df
    # Replace all strings with "none"
df_numeric = df.replace(r'\D', "", regex=True)
    # Change columns type from object to numeric
df_numeric = df_numeric.apply(pd.to_numeric, errors='coerce')
df_numeric

,,R1_2019,R2_2019,EOS_2019,MID_2019,R1_2020,R2_2020,EOS_2020,MID_2020,R1_2021,R2_2021
CODE,TITLE,,,,,,,,,,
AC120,International Business,234.0,234.0,234.0,269.0,209.0,NaN,209.0,280.0,294.0,294.0
AC137,Liberal Arts,275.0,252.0,252.0,275.0,252.0,NaN,252.0,270.0,271.0,270.0
AD101,First Year Art & Design (Common Entry),NaN,NaN,550.0,NaN,NaN,NaN,NaN,NaN,554.0,NaN
AD102,Graphic Design and Moving Image Design,NaN,NaN,635.0,NaN,NaN,NaN,NaN,NaN,538.0,NaN
AD103,Textile & Surface Design and Jewellery & Objects,NaN,NaN,545.0,NaN,NaN,NaN,NaN,NaN,505.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
WD211,Creative Computing,275.0,NaN,275.0,322.0,271.0,NaN,271.0,318.0,270.0,NaN
WD212,Recreation and Sport Management,274.0,NaN,274.0,311.0,270.0,NaN,270.0,349.0,262.0,NaN
WD230,Mechanical and Manufacturing Engineering,273.0,NaN,273.0,348.0,253.0,NaN,253.0,369.0,230.0,230.0


In [27]:
df_numeric.to_csv ("Numeric_table.csv")

In [28]:
df_numeric.dtypes

R1_2019     float64
R2_2019     float64
EOS_2019    float64
MID_2019    float64
R1_2020     float64
R2_2020     float64
EOS_2020    float64
MID_2020    float64
R1_2021     float64
R2_2021     float64
dtype: object

In [29]:
# 2019
r1_2019 = df_numeric ["R1_POINTS_2019"]
r2_2019 = df_numeric ["R2_POINTS_2019"]
eos_2019 = df_numeric ["EOS_2019"]
mid_2019 = df_numeric ["MID_2019"]
# 2020
r1_2020 = df_numeric ["R1_POINTS_2020"]
r2_2020 = df_numeric ["R2_POINTS_2020"]
eos_2020 = df_numeric ["EOS_2020"]
mid_2020 = df_numeric ["MID_2020"]
# 2021
r1_2021 = df_numeric ["R1_POINTS_2021"]
r2_2021 = df_numeric ["R2_POINTS_2021"]

KeyError: 'R1_POINTS_2019'

In [ ]:
r1_2019

In [ ]:
desc = df_numeric.describe()
desc

In [ ]:
df_numeric.loc[df_numeric['R1_POINTS_2019'] == 59]

In [ ]:
df_numeric["R1_POINTS_2020"].value_counts()

In [ ]:
desc.plot.bar()

In [ ]:
df_numeric.corr()

In [ ]:
plt.rcParams['figure.figsize'] = [25, 10]
sns.heatmap (data = df_numeric.corr(), square = True, annot = True, cmap = "mako")
plt.xticks (rotation = 45)
plt.yticks (rotation = 45)
plt.show()

In [ ]:
plt.xticks (ticks = (100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100))
plt.hist ([r1_2019, r1_2020, r1_2021], alpha = 0.7)
plt.show()

In [ ]:
new = pd.read_csv ("Numeric_table.csv")
c = new [["MID_2019", "MID_2020"]]

In [ ]:
sns.scatterplot (data = c)
plt.show()

In [ ]:
sns.boxplot (data = [r1_2019, r1_2020, r1_2021])
plt.show()

In [ ]:
sns.kdeplot (data = [r1_2019, r1_2020, r1_2021])
plt.show()

### DATA COMPARISON

# CONCLUSION

***

## REFERENCES
***

- [1] http://www2.cao.ie/handbook/handbook2022/hb.pdf
- [2] https://en.wikipedia.org/wiki/Central_Applications_Office7
- [3] https://www.independent.ie/life/family/learning/understanding-your-cao-course-guide-26505318.html
- [4] https://en.wikipedia.org/wiki/Regular_expression

## TESTING

In [ ]:
df_numeric.dropna(axis = 0, how = 'all', inplace = True)
df_numeric

In [ ]:
df2019_num_r1 = df ["R1_POINTS_2019"]
df2019_num_r2 = df_numeric ["R2_POINTS_2019"]
df2019_num_eos = df_numeric ["EOS_2019"]
df2019_num_mid = df_numeric ["MID_2019"]

df2020_num_r1 = df_numeric ["R1_POINTS_2020"]
df2020_num_r2 = df_numeric ["R2_POINTS_2020"]
df2020_num_eos = df_numeric ["EOS_2020"]
df2020_num_mid = df_numeric ["MID_2020"]

df2021_num_r1 = df_numeric ["R1_POINTS_2021"]
df2021_num_r2 = df_numeric ["R2_POINTS_2021"]

In [ ]:
df2019_num_r1

In [ ]:
a = df2019_num_r1[df2019_num_r1.str.contains("\*", na = False)]
a

In [ ]:
b = a['R1_POINTS_2019'].str.split('[0-9]').str[-1]
b

In [ ]:
b.count()

|||
| :-: | :- |
| * | Not all on this points score were offered places |
| # | Test / Interview / Portfolio / Audition |
| AQA | All qualified applicants |
| v | New competition for available places |